In [ ]:
import os
import capytaine as cpy
import autograd.numpy as np
import numpy as nnp
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import brute

import wecopttool as wot
wot.set_loglevel('INFO')

## WEC object
We will go through a number of steps to create a `WEC` object, with which we can solve for the optimal control trajectory.
To compose the `WEC` object, we will need to define the mesh, degrees of freedom, mass and hydrostatic properties, linear hydrodynamic coefficients (from a BEM solution), any additional dynamic forces (e.g. PTO force), and constraints (e.g. minimum line tension).  

### Geometry
First we create a surface mesh for the WEC hull, and quickly visualize the cross-sectional shape of the hull.
The AquaHarmonics hull mesh is already included with WecOptTool—take a look at the `geometry.py` module if you're interested in seeing how it was created.
Note that the lower cylindrical section of the hull is open to the sea.

In [ ]:
hull_geom = wot.geom.WaveBot(r1=3.302/2,
                             r2=3.302/2*0.875, 
                             h1=0.2,
                             h2=0.3,
                             scale_factor=1)  # use standard dimensions
mesh = hull_geom.mesh(mesh_size_factor=0.5)
_ = hull_geom.plot_cross_section()

Next we create a `FloatingBody` object from the mesh in Capytaine, which we will use to calculate the hydrodynamics.
We can visualize a 3D rendering of the mesh now as well.

In [ ]:
fb = cpy.FloatingBody.from_meshio(mesh, name="Pioneer")
fb.add_rotation_dof
# fb.add_translation_dof(name="Heave")
fb.add_rotation_dof(axis=cpy.meshes.geometry.Axis((0, 1, 0), point=(0, 0, 0.1)),
                    name='Pitch')
fb.add_rotation_dof(axis=cpy.meshes.geometry.Axis((0, 1, 0), point=(0, 0, 0.1)),
                    name='Motor shaft',
                    amplitude=0)
ndof = fb.nb_dofs
fb.show_matplotlib()

### Hydrodynamics and hydrostatics

Next we will run the boundary element model (BEM) [Capytaine](https://github.com/capytaine/capytaine) to obtain the hydrodynamic coefficients for the hull.
Before running the BEM solver, we must specify a set of frequencies at which to perform the calculations.
For `WecOptTool`, these must be a regularly spaced frequency array. 
We will also save the results to a file so that they can be used later, in Part 2, without re-running the BEM solver.

In [ ]:
g = 9.81
rho = 1025

f1 = 0.0125 # Hz
nfreq = 100
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

file_name = "PioneerWec_hydro.nc"
if os.path.exists(file_name):
    bem_data = wot.read_netcdf(file_name)
else:    
    bem_data = wot.run_bem(fb, freq, rho=rho, g=g)
    wot.write_netcdf(file_name, bem_data)
    
bem_data['frequency'] = bem_data.omega/2/np.pi
bem_data['period'] = 1 / bem_data.frequency
bem_data = bem_data.set_coords(['frequency','period'])
bem_data['frequency'].attrs['long_name'] = 'Frequency'
bem_data['frequency'].attrs['units'] = 'Hz'
bem_data['period'].attrs['long_name'] = 'Period'
bem_data['period'].attrs['units'] = 's'


displaced_mass = wot.hydrostatics.inertia_matrix(fb, rho).values  # kg
displacement = displaced_mass/rho # m^3 

flywheel_inertia = 4.5 # kg-m^2 TODO: check value
flywheel_spring = 600. # N-m
flywheel_coulomb_fr = 2. # N-m
flywheel_visc_fr = 0.02 # N-m/rad

hydro = wot.linear_hydrodynamics(bem_data, 
                                 inertia_matrix=np.array([[7484,0],[0, flywheel_inertia]]), 
                                 hydrostatic_stiffness=np.array([[58e3, flywheel_spring],[flywheel_spring, 0]]), 
                                 friction=np.array([[1e3,flywheel_friction],[flywheel_friction,0]])
                                 )
hydro = wot.check_linear_damping(hydro)
Zi = wot.hydrodynamic_impedance(hydro)

In [ ]:
# Excitation
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

Hexc = (bem_data['Froude_Krylov_force'] + bem_data['diffraction_force']).squeeze().sel(influenced_dof='Pitch')

20*np.log10(np.abs(Hexc)).plot(x='frequency', ax=ax[0])

ax[1].plot(Hexc.frequency, 180/np.pi*np.angle(Hexc))

for axi in ax:
    axi.label_outer()
    axi.set_title('')
    axi.grid()
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)

ax[0].set_ylabel('Mag. [dB]')
ax[1].set_ylabel('Angle [deg]')

ax[0].set_title('Excitation')

# fig.savefig(os.path.join(gfx_path, 'PioneerWec_hydro_excitation.pdf'))


# Impedance, Bode
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

_ = 20*np.log10(np.abs(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze())).plot(ax=ax[0],
                                           x='frequency')
ax[1].plot(Zi.frequency, np.angle(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze())*180/np.pi)

for axi in ax:
    axi.label_outer()
    axi.set_title('')
    axi.grid()
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    
ax[0].set_ylabel('Mag. [dB]')
ax[1].set_ylabel('Angle [deg]')

ax[0].set_title('Impedance')

# fig.savefig(os.path.join(gfx_path, 'PioneerWec_hydro_impedance_Bode.pdf'))

# Impedance, real/imag.
fig, ax = plt.subplots(nrows=2,
                       sharex=True)



np.real(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze()).plot(ax=ax[0], x='frequency')
np.imag(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze()).plot(ax=ax[1], x='frequency')

ax[0].set_ylabel('$\\Re\\{Z_i\\}$')
ax[1].set_ylabel('$\\Im\\{Z_i\\}$')

for axi in ax:
    axi.label_outer()
    axi.set_title('')
    axi.grid()
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    
ax[0].set_title('Impedance')

### PTO object
Using both the parametric kinematics (gear ratios) and the motor power loss map, we can now create a PTO object that will be inserted in our WEC object.
Note that the friction and inertia effects will be included as additional forces.
By defining the PTO object in this way, we will be able to more clearly distinguish the power generating forces from other effects (e.g., friction).

In [ ]:
# omega = bem_data.omega.values 
def gear_ratio(max_spring_trq, moi, t):
    f = 2*np.pi / t
    k_spring = max_spring_trq / (np.pi/4)
    return np.sqrt(k_spring / (moi * f**2))

torque_constant = 6.7
winding_resistance = 0.5
winding_inductance = 0.0
drivetrain_inertia = 0  # TODO - already done via second body
drivetrain_friction = 0
drivetrain_stiffness = 0

# drivetrain_impedance = (1j*omega*drivetrain_inertia +
#                         drivetrain_friction +
#                         1/(1j*omega)*drivetrain_stiffness)

# winding_impedance = winding_resistance + 1j*omega*winding_inductance


# pto_impedance_11 = -1 * gear_ratio**2 * drivetrain_impedance
# off_diag = np.sqrt(3.0/2.0) * torque_constant * gear_ratio
# pto_impedance_12 = -1*(off_diag+0j) * np.ones(omega.shape)
# pto_impedance_21 = -1*(off_diag+0j) * np.ones(omega.shape)
# pto_impedance_22 = winding_impedance
# pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
#                           [pto_impedance_21, pto_impedance_22]])


def power_loss(speed, torque, winding_resistance=0.5, torque_coefficient=6.7):
    return winding_resistance * (torque / torque_coefficient)**2

# name = ["PTO_Heave",]
# kinematics = gear_ratio_generator*np.eye(ndof)
# controller = None
nstate_opt = 2*nfreq
pto = wot.pto.PTO(1,
                  kinematics = np.array([[1, -1/gear_ratio],]), #TODO
                #   impedance=pto_impedance,
                  loss=power_loss,
                  )

# def f_spring(wec, x_wec, x_opt, waves, nsubsteps=1, k_spring=50):
#     pos = wec.vec_to_dofmat(x_wec)
#     time_matrix = wec.time_mat_nsubsteps(nsubsteps)
#     spring = -1 * k_spring * pos
#     return np.dot(time_matrix, spring)

# def f_friction(wec, x_wec, x_opt, waves, nsubsteps=1, b_friction=1e-2):
#     pos = wec.vec_to_dofmat(x_wec)
#     vel = np.dot(wec.derivative_mat, pos)
#     time_matrix = wec.time_mat_nsubsteps(nsubsteps)
#     fric = -1 * b_friction * vel
#     return np.dot(time_matrix, fric)

# def f_inertia(wec, x_wec, x_opt, waves, nsubsteps=1, inertia_pto=50*0.5**2):
#     pos = wec.vec_to_dofmat(x_wec)
#     vel = np.dot(wec.derivative_mat,pos)
#     acc = np.dot(wec.derivative_mat, vel)
#     time_matrix = wec.time_mat_nsubsteps(nsubsteps)
#     inertia = inertia_pto * acc
#     f_inertia = np.dot(time_matrix,inertia)
#     return f_inertia

def const_stroke_pto(wec, x_wec, x_opt, waves, nsubsteps=2, stroke_max=15): 
    pos = pto.position(wec, x_wec, x_opt, waves, nsubsteps)
    return stroke_max - np.abs(pos.flatten())

def const_peak_torque_pto(wec, x_wec, x_opt, waves, nsubsteps=2, torque_peak_max=75):
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

# def const_power_pto(wec, x_wec, x_opt, waves, nsubsteps=2, max_power=0):
#     power_mech = pto.mechanical_power(wec, x_wec, x_opt, waves, nsubsteps)
#     return max_power - power_mech.flatten()

### Create object
Finally, we can use all the different components we've developed thus far to construct a `WEC` object:

 * **BEM data** - defines the hydrodynamics of the hull
 * **Inertia and hydrostatics** - rigid-body inertia and hydrostatic stiffness
 * **Constraints** - limitations on the hardware (max power, max torque, etc.) and the constraint to prevent the tether from going slack
 * **Additional forces** - this captures all of the forces on the WEC that are not due to the interaction between the hull and water (PTO, etc.)

In [ ]:
wec = wot.WEC.from_impedance(freqs=Zi.omega.values/2/np.pi,
                             impedance=Zi,
                             exc_coeff=hydro['Froude_Krylov_force'] +
                             hydro['diffraction_force'],
                             hydrostatic_stiffness=hydro['hydrostatic_stiffness'],
                             f_add={'Generator': pto.force_on_wec,
                                    # 'Spring': f_spring,
                                    # 'Friction': f_friction,
                                    # 'Inertia': f_inertia,
                                    },
                             constraints=[
                                #  {'type': 'ineq', 'fun': const_power_pto, },
                                #  {'type': 'ineq', 'fun': const_peak_torque_pto, },
                                #  {'type': 'ineq', 'fun': const_buoy_pos, },
                             ]
                             )

wec._dof_names = ['Pitch', 'Motor shaft']


## Waves
A regular wave will allow us to get a good initial understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = 1.6/2
# wavefreq =  1/5.3
wavefreq =  0.3875 # TODO
phase = 0
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)
# spectrum = lambda f: wot.waves.jonswap_spectrum(freq=f, fp=wavefreq, hs=2*amplitude)
# efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
# waves = wot.waves.long_crested_wave(efth)
# efth.plot(marker='.')

In [ ]:
fig, ax = plt.subplots()
(np.abs(waves)/np.max(np.abs(waves))).plot(marker='.', label='Waves')
Zi_mag = np.abs(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze())
(Zi_mag/np.max(Zi_mag)).plot(label='Impedance')

ax.autoscale(axis='x', tight=True)
ax.set_ylim((0,1))
ax.set_title('')
ax.legend()

## Solve
Finally, we solve for the optimal control trajectory.
To ensure the numerical optimization problem is well-scaled, we set [specific scaling factors](https://snl-waterpower.github.io/WecOptTool/theory.html#scaling) for the WEC position (`scale_x_wec`), the PTO force (`scale_x_opt`), and objective function (`scale_obj`).
We will also set the [maximum iteration and objective function tolerance for the optimization solver](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html).
This step typically takes about 30s.

In [ ]:

# def my_obj_fun(wec, x_wec, x_opt, waves, steps=1):
#     return np.abs(pto.average_power(wec, x_wec, x_opt, waves, steps))

obj_fun = pto.average_power

results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    x_wec_0=np.ones(wec.nstate_wec) * 1e-1,
    x_opt_0=np.ones(nstate_opt) * 1e-1,
    optim_options={
        # 'maxiter': 200, 
        # 'ftol': 1e-8,
        },
    # scale_x_wec=1e1,
    # scale_x_opt=1e3,
    # scale_obj=1e-3,
)

## Post-process and plotting
Next, we post-process the results to allow us to more easily visualize them in a series of plots.
Note that because the `WEC` and `PTO` objects are distinct (the WEC object really only knows what the force from the PTO is, not how how that force is obtained), we create two separate results objects (in each case, we get results in the frequency domain and time domain).

In [ ]:
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=5)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps=5)
pto_tdom.power.mean()
print(f'Optimal average power: {results.fun:.2f} W')

In [ ]:
# Fex = wec_fdom.force.sel(type=['excitation']).sum('type').squeeze().sel(influenced_dof='Pitch')
# P_max = np.abs(Fex)**2 / (8*np.real(Zi.squeeze().sel(influenced_dof='Pitch', radiating_dof='pitch')))
# fig, ax = plt.subplots()
# P_max.plot()
# ax.set_title('$P_{max}=$' + f'{P_max.sum().item()/1e3:.1f} kW')

In [ ]:
fig, ax = plt.subplots(nrows=4, sharex=True, figsize=(12, 12))

# Excitation
force_excitation = wec_tdom.force.sel(type=['excitation'])
force_excitation.sel(influenced_dof='Pitch').plot(ax=ax[0], color='k', label='Excitation')
ax[0].set_ylabel('Exc. torque [Nm]')
ax0 = ax[0].twinx()
pto_tdom.force.plot(ax=ax0, label='Motor', color='red')
ax0.set_ylabel('PTO torque [Nm]', color='red')
ax0.tick_params(axis='y', labelcolor='red')
ax0.set_title('')
ax0.autoscale(enable=True, axis='x', tight=False)

(wec_tdom.pos*180/np.pi).plot(ax=ax[1], hue='influenced_dof')
ax[1].set_ylabel('Position [deg]')

wec_tdom.vel.plot(ax=ax[2], hue='influenced_dof', add_legend=False)

(pto_tdom['mech_power']).plot(ax=ax[3], 
                                  label='Mech. power ($\\bar{P}_{mech}$: ' + f'{pto_tdom.mech_power.mean().item():.0f} W)')
(pto_tdom['power']).plot(ax=ax[3], linestyle='dashed', 
                         label='Elec. power ($\\bar{P}_{elec}$: ' + f'{pto_tdom.power.mean().item():.0f} W)')

ax[3].legend(loc='upper right')
ax[3].set_ylabel('Power [W]')

for axi in ax:
    axi.set_title('')
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)

# # fig.savefig('../gfx/PioneerWec_wecopttool_time_histories.pdf')